# Introduction to Pathogen.jl

## Initialization

We start by loading Pathogen.jl in julia 1.0

In [ ]:
# These steps are optional if you'd like to run 
using Pkg
Pkg.activate(joinpath(@__DIR__, "../"))
using Pathogen

Additionally we'll use the core Packages:
* `Random` for setting the random seed for replicability
* `Distances` for various efficienct distance calculation methods
* `DataFrames` for storing individual level information
* `Distributions` for random generation of individual level information
* `Plots` for visualization

In [ ]:
using Random, Distances, DataFrames, Distributions, Plots

## Simulation
We'll set the seed for random number generation such that our results here are reproducible:

In [ ]:
Random.seed!(5432)

We'll define a population consisting of *n* individuals, uniformaly distributed over a 10x10 unit area. For our population we'll assume to have information on a single risk factor, which we'll distribute following a Gamma(1,1) distribution.

With Pathogen.jl, the `Population` type contains all population information, it consists of:
* `risks` - an optional `DataFrame` with all individual level information (risk factors, location)
* `distances` - an optional Array{T<:Any, 2} with precalculated distances
* `individuals` - the number of individuals in the population

In [ ]:
n = 250

risks = DataFrame(x = rand(Uniform(0, 10), n),
                  y = rand(Uniform(0, 10), n),
                  riskfactor1 = rand(Gamma(), n))

pop = Population(risks)

# Will precalculate distances
pop.distances = [euclidean([risks[:x][i]; risks[:y][i]], [risks[:x][j]; risks[:y][j]]) for i = 1:pop.individuals, j = 1:pop.individuals]


Pathogen.jl supports SEIR, SEI, SIR, and SI individual level models. Within each of those model types, the user has full control over the functions describing individual specific transition rates between disease states (*i.e.* form of, and relevant risk factors to, susceptibility, infectivity, transmissability, latency, removal, and sparks functions when applicable). For ease of use, some common forms of these functions are provided in `examples/risk_functions.jl` which can either be loaded and used directly, or serve as examples to help the user construct their own. For this example we will use some of these functions directly in an SEIR individual level model. 

In [ ]:
include(joinpath(@__DIR__, "risk_functions.jl"))

rf = RiskFunctions{SEIR}(_constant, # sparks function
                         _coefficient, # susceptibility function
                         _powerlaw, # transmissability function
                         _one, # infectivity function
                         _constant, # latency function
                         _constant) # removal function

Each of these functions is parameterized by a `Vector{Float64}`...

In [ ]:
rparams = RiskParameters{SEIR}([0.001], # sparks function parameter(s)
                               [1.0], # susceptibility function parameter(s)
                               [1.0, 6.0], # transmissibility function parameter(s)
                               Float64[], # infectivity function parameter(s)
                               [0.1], # latency function parameter(s)
                               [0.05]) # removal function parameter(s)

We initialize an Epidemic `Simulation` object by supplying the population `DataFrame` with our individual level model risk functions and their associated parametrizations.

In [ ]:
sim = Simulation(pop, rf, rparams)

This simulation can now be ran until either a specified stop condition is met (computation time, simulation time, or a maximum number of iterations), or until there are no further possible events.

In [ ]:
simulate!(sim, tmax=100.0)

### Visualization
Using our preferred plotting backend we can produce several informative plots for a simulated epidemic...

In [ ]:
gr()

a `plot()` method is provided for `Events` objects - such as those generated with an epidemic simulation. This will plot an epidemic curve.

In [ ]:
plot(sim.events)
png(joinpath(@__DIR__, "01_epidemic_curve.png"))

We can also visualize transmission networks through time - below is an animation showing the spread of an epidemic for the first 100 time units.

In [ ]:
epidemic_animation = @animate for t in range(0.0, stop=100.0, length=100) # From t = 0.0 to t= 100.0 with 100 increments
    plot(sim.transmission_network, sim.population, sim.events, t)
end
mp4(epidemic_animation, joinpath(@__DIR__, "01_epidemic_animation.mp4"), fps=10)